In [ ]:
# !unzip '/content/drive/MyDrive/journey-springfield.zip'

In [2]:
import PIL

import pickle
import numpy as np
from skimage import io

from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path

import torchvision
from torchvision import transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau, OneCycleLR
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.optim import Adam, SGD

from matplotlib import colors, pyplot as plt
%matplotlib inline

# в sklearn не все гладко, чтобы в colab удобно выводить картинки 
# мы будем игнорировать warnings
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

In [3]:
import os
import torch
import random
from PIL import Image
import wandb
import copy

In [4]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from albumentations.augmentations.blur import transforms

In [5]:
# since v0.13 parameter pretrained is depricated and will be removed in v0.15

torchvision.__version__

'0.12.0'

In [6]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
def seed_everything(seed):
    # Фискирует максимум сидов.
    # Это понадобится, чтобы сравнение оптимизаторов было корректным
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything(4242)

In [8]:
# разные режимы датасета 
DATA_MODES = ['train', 'val', 'test']
# все изображения будут масштабированы к размеру 224x224 px
RESCALE_SIZE = 224
# работаем на видеокарте
DEVICE = torch.device("cuda")

In [9]:
class SimpsonsDataset(Dataset):
    """
    Датасет с картинками, который паралельно подгружает их из папок
    производит скалирование и превращение в торчевые тензоры
    """
    def __init__(self, files, mode, transform):
        super().__init__()
        self.transform = transform
        # список файлов для загрузки
        self.files = sorted(files)
        # режим работы
        self.mode = mode

        if self.mode not in DATA_MODES:
            print(f"{self.mode} is not correct; correct modes: {DATA_MODES}")
            raise NameError

        self.len_ = len(self.files)
     
        self.label_encoder = LabelEncoder()

        if self.mode != 'test':
            self.labels = [path.parent.name for path in self.files]
            self.label_encoder.fit(self.labels)

            with open('label_encoder.pkl', 'wb') as le_dump_file:
                  pickle.dump(self.label_encoder, le_dump_file)
                      
    def __len__(self):
        return self.len_
      
    def load_sample(self, file):
        image = Image.open(file)
        image.load()
        return image
  
    def __getitem__(self, index):
        x = self.load_sample(self.files[index])
        x = np.array(x, dtype='float32')
        x = self.transform(image=x)['image']
        if self.mode == 'test':
            return x
        else:
            label = self.labels[index]
            label_id = self.label_encoder.transform([label])
            y = label_id.item()
            return x, y

In [8]:
# augmentations
train_transform = A.Compose(
    [
        A.SmallestMaxSize(max_size=160),
        transforms.Blur(blur_limit=7, always_apply=False, p=0.5),
        A.augmentations.transforms.Downscale(scale_min=0.5, scale_max=0.5, interpolation=None, always_apply=False, p=0.5),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=3, p=0.3),
        A.RandomCrop(height=128, width=128),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ]
)

val_transform = A.Compose(
    [
        A.SmallestMaxSize(max_size=160),
        A.CenterCrop(height=128, width=128),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)

/opt/conda/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:1555: UserWarning: Using default interpolation INTER_NEAREST, which is sub-optimal.Please specify interpolation mode for downscale and upscale explicitly.For additional information see this PR https://github.com/albumentations-team/albumentations/pull/584
  "Using default interpolation INTER_NEAREST, which is sub-optimal."


In [9]:
def imshow(inp, title=None, plt_ax=plt, default=False):
    """Imshow для тензоров"""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt_ax.imshow(inp)
    if title is not None:
        plt_ax.set_title(title)
    plt_ax.grid(False)

In [10]:
TRAIN_DIR = Path('../input/dddddd/journey-springfield/train/simpsons_dataset')
TEST_DIR = Path('../input/dddddd/journey-springfield/testset/testset')

train_val_files = sorted(list(TRAIN_DIR.rglob('*.jpg')))
test_files = sorted(list(TEST_DIR.rglob('*.jpg')))

In [11]:
from sklearn.model_selection import train_test_split

train_val_labels = [path.parent.name for path in train_val_files]
train_files, val_files = train_test_split(train_val_files, test_size=0.25, \
                                          stratify=train_val_labels)

In [12]:
val_dataset = SimpsonsDataset(val_files, mode='val', transform=val_transform)
train_dataset = SimpsonsDataset(train_files, mode='train', transform=train_transform)

NameError: name 'SimpsonsDataset' is not defined

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=3,figsize=(8, 8), \
                        sharey=True, sharex=True)
for fig_x in ax.flatten():
    random_characters = int(np.random.uniform(0,1000))
    im_val, label = train_dataset[random_characters]
    img_label = " ".join(map(lambda x: x.capitalize(),\
                train_dataset.label_encoder.inverse_transform([label])[0].split('_')))
    imshow(im_val.data.cpu(), \
          title=img_label,plt_ax=fig_x)

In [ ]:
from torchvision.models import resnet34

In [ ]:
model = resnet34(pretrained=False)

In [ ]:
model.fc = nn.Linear(512, 42)

In [ ]:
len(train_dataset)

In [29]:
def fit_epoch(model, train_loader, criterion, optimizer, cycle, scheduler):
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
  
    for inputs, labels in train_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        preds = torch.argmax(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_data += inputs.size(0)
        
        if cycle:
            scheduler.step()
              
    train_loss = running_loss / processed_data
    train_acc = running_corrects.cpu().numpy() / processed_data
    return train_loss, train_acc

In [30]:
def eval_epoch(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    processed_size = 0

    for inputs, labels in val_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            preds = torch.argmax(outputs, 1)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_size += inputs.size(0)
    val_loss = running_loss / processed_size
    val_acc = running_corrects.double() / processed_size
    return val_loss, val_acc

In [21]:
def train(train_files, val_files, model, epochs, batch_size, scheduler):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f}"
    
    #checkpoint = torch.load('/kaggle/working/checkpoint_3.2.7')

    with tqdm(desc="epoch", total=epochs) as pbar_outer:
        opt = torch.optim.Adam(model.parameters(), lr = 6.111e-7, weight_decay=0.001)
        #opt.load_state_dict(checkpoint['optimizer_state_dict'])
        criterion = nn.CrossEntropyLoss()
        sch = scheduler
        #sch.load_state_dict(checkpoint['scheduler_state_dict'])
        for epoch in range(epochs):
            train_loss, train_acc = fit_epoch(model, train_loader, criterion, opt, sch)
            print("loss", train_loss)
            
            val_loss, val_acc = eval_epoch(model, val_loader, criterion)
            lr = opt.param_groups[0]['lr']
            #sch.step(val_loss)
            history.append((train_loss, train_acc, val_loss, val_acc))
            
            wandb.log({'train_loss': train_loss, 'train_acc': train_acc, 'val_loss': val_loss, 
                       'val_acc': val_acc, 'lr': lr}, step = epoch)
            
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss,\
                                           v_loss=val_loss, t_acc=train_acc, v_acc=val_acc))
            
        torch.save({'optimizer_state_dict': opt.state_dict(), 'scheduler_state_dict': sch.state_dict()}, 'checkpoint_3.2.7')
            
    return history

In [22]:
def predict(model, test_loader):
    with torch.no_grad():
        logits = []
    
        for inputs in test_loader:
            inputs = inputs.to(DEVICE)
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)
            
    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

In [ ]:
checkpoint = torch.load('/kaggle/input/dddddd/model_exp_3.2.4')

In [ ]:
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
model = model.to(DEVICE)

In [ ]:
for param in model.parameters():
    param.requred_grad = True

In [ ]:
config = {
    'model': 'resnet_34',
    'pretrained': True,
    'optimizer': 'Adam',
    'scheduler': 'OneCycleLR',
    'n_epochs': 10,
    'batch_size': 64,
    'start_lr': 6.111e-7,
    'full_train': True,
    'weight_decay': None,
    'patience': None
}

In [ ]:
id_ = wandb.util.generate_id()

In [ ]:
run = wandb.init(
    id = id_,
    project = 'simpsons',
    name = 'exp_3.2.11',
    reinit = True,
    config = config
)

history = train(train_dataset, val_dataset, model=model, epochs=10, batch_size=64)



In [ ]:
#wandb.finish()

In [ ]:
torch.save({
    'model_state_dict': model.state_dict()
}, 'model_exp_3.2.11')

In [ ]:
loss, acc, val_loss, val_acc = zip(*history)

In [ ]:
plt.figure(figsize=(15, 9))
plt.plot(loss, label="train_loss")
plt.plot(val_loss, label="val_loss")
plt.legend(loc='best')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [1]:
def predict_one_sample(model, inputs, device=DEVICE):
    """Предсказание, для одной картинки"""
    with torch.no_grad():
        inputs = inputs.to(device)
        model.eval()
        logit = model(inputs).cpu()
        probs = torch.nn.functional.softmax(logit, dim=-1).numpy()
    return probs

NameError: name 'DEVICE' is not defined

In [ ]:
random_characters = int(np.random.uniform(0,1000))
ex_img, true_label = val_dataset[random_characters]
probs_im = predict_one_sample(model, ex_img.unsqueeze(0))

In [ ]:
idxs = list(map(int, np.random.uniform(0,1000, 20)))
imgs = [val_dataset[id][0].unsqueeze(0) for id in idxs]

probs_ims = predict(model, imgs)

In [ ]:
label_encoder = pickle.load(open("label_encoder.pkl", 'rb'))

In [ ]:
y_pred = np.argmax(probs_ims,-1)

actual_labels = [val_dataset[id][1] for id in idxs]

preds_class = [label_encoder.classes_[i] for i in y_pred]

In [ ]:
from sklearn.metrics import f1_score

f1_score(actual_labels, y_pred, average='micro')

In [ ]:
import matplotlib.patches as patches
from matplotlib.font_manager import FontProperties

fig, ax = plt.subplots(nrows=3, ncols=3,figsize=(12, 12), \
                        sharey=True, sharex=True)
for fig_x in ax.flatten():
    random_characters = int(np.random.uniform(0,1000))
    im_val, label = val_dataset[random_characters]
    img_label = " ".join(map(lambda x: x.capitalize(),\
                val_dataset.label_encoder.inverse_transform([label])[0].split('_')))
    
    

    imshow(im_val.data.cpu(), \
          title=img_label,plt_ax=fig_x)
    
    actual_text = "Actual : {}".format(img_label)
            
    fig_x.add_patch(patches.Rectangle((0, 53),86,35,color='white'))
    font0 = FontProperties()
    font = font0.copy()
    font.set_family("fantasy")
    prob_pred = predict_one_sample(model, im_val.unsqueeze(0))
    predicted_proba = np.max(prob_pred)*100
    y_pred = np.argmax(prob_pred)
    
    predicted_label = label_encoder.classes_[y_pred]
    predicted_label = predicted_label[:len(predicted_label)//2] + '\n' + predicted_label[len(predicted_label)//2:]
    predicted_text = "{} : {:.0f}%".format(predicted_label,predicted_proba)
            
    fig_x.text(1, 59, predicted_text , horizontalalignment='left', fontproperties=font,
                    verticalalignment='top',fontsize=8, color='black',fontweight='bold')

# Resnet50

In [62]:
from torchvision.models import resnet50

In [63]:
model = resnet50(pretrained=True)
for param in model.parameters():
    param.requred_gred = False
    
model.fc = nn.Linear(2048, 42)

In [64]:
model.to(DEVICE)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [33]:
transforms_without_aug = A.Compose(
    [
        A.SmallestMaxSize(max_size=160),
        A.CenterCrop(height=128, width=128),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)

In [35]:
val_dataset = SimpsonsDataset(val_files, mode='val', transform=transforms_without_aug)
train_dataset = SimpsonsDataset(train_files, mode='train', transform=transforms_without_aug)

In [36]:
scheduler_red = torch.optim.lr_scheduler.ReduceLROnPlateau
scheduler_cyc = torch.optim.lr_scheduler.OneCycleLR

# ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)
# OneCycleLR(optimizer, max_lr=0.00001, total_steps=None, epochs=epochs, steps_per_epoch=len(train_loader))

In [57]:
def train(train_files, val_files, model, epochs, batch_size):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f}"
    
    #checkpoint = torch.load('/kaggle/working/checkpoint_3.2.7')

    with tqdm(desc="epoch", total=epochs) as pbar_outer:
        opt = torch.optim.Adam(model.parameters(), lr = 0.00001, weight_decay=0.001)
        #opt.load_state_dict(checkpoint['optimizer_state_dict'])
        criterion = nn.CrossEntropyLoss()
        sch = scheduler_red(opt, patience=3)
        #sch.load_state_dict(checkpoint['scheduler_state_dict'])
        for epoch in range(epochs):
            train_loss, train_acc = fit_epoch(model, train_loader, criterion, opt, False, sch)
            print("loss", train_loss)
            
            val_loss, val_acc = eval_epoch(model, val_loader, criterion)
            lr = opt.param_groups[0]['lr']
            sch.step(val_loss)
            history.append((train_loss, train_acc, val_loss, val_acc))
            
            wandb.log({'train_loss': train_loss, 'train_acc': train_acc, 'val_loss': val_loss, 
                       'val_acc': val_acc, 'lr': lr}, step = epoch)
            
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss,\
                                           v_loss=val_loss, t_acc=train_acc, v_acc=val_acc))
            
        torch.save({'optimizer_state_dict': opt.state_dict(), 'scheduler_state_dict': sch.state_dict(),
                    'model_state_dict': model.state_dict()}, 'checkpoint_4.0.0')
            
    return history

In [68]:
config = {
    'model': 'resnet_50',
    'pretrained': True,
    'optimizer': 'Adam',
    'scheduler': 'ReduceLROnPlateau',
    'n_epochs': 10,
    'batch_size': 64,
    'start_lr': 0.0001,
    'full_train': True,
    'weight_decay': 0.00001,
    'patience': 3
}

In [69]:
id_ = wandb.util.generate_id()

In [70]:
run = wandb.init(
    id = id_,
    project = 'simpsons',
    name = 'exp_4.0.1',
    reinit = True,
    config = config
)

history = train(train_dataset, val_dataset, model=model, epochs=10, batch_size=64)



lr,▁▁▁▁▁▁▁▁▁▁
train_acc,▁▆▇▇██████
train_loss,█▃▂▂▁▁▁▁▁▁
val_acc,▁▆▇▇▇█████
val_loss,█▃▂▂▁▁▁▁▁▁
lr,1e-05
train_acc,0.94586
train_loss,0.20111
val_acc,0.95281
val_loss,0.20273


epoch:   0%|          | 0/10 [00:00<?, ?it/s]

loss 0.18907331144937292


epoch:  10%|█         | 1/10 [03:00<27:02, 180.30s/it]


Epoch 001 train_loss: 0.1891     val_loss 0.2003 train_acc 0.9483 val_acc 0.9530
loss 0.17775125097373293


...

epoch:  90%|█████████ | 9/10 [24:03<02:33, 153.76s/it]


Epoch 009 train_loss: 0.1094     val_loss 0.1906 train_acc 0.9690 val_acc 0.9593
loss 0.09618936007444596


epoch: 100%|██████████| 10/10 [26:29<00:00, 151.52s/it]


Epoch 010 train_loss: 0.0962     val_loss 0.1741 train_acc 0.9722 val_acc 0.9624


epoch: 100%|██████████| 10/10 [26:30<00:00, 159.06s/it]


In [67]:
for param in model.parameters():
    param.required_grad = True